In [1]:
import pandas as pd
import os
import geopandas as gpd
import json
from shapely.geometry import box
import numpy as np
from functools import reduce
import time
import urllib.request
import zipfile
import requests

In [22]:
ActionDate = time.strftime('%Y%m%d')
# newItemscsv = os.path.join('reports', f'allNewItems_{ActionDate}.csv')
newItemscsv = os.path.join('reports', 'shp.csv')
df_csv = pd.read_csv(newItemscsv, encoding = 'unicode_escape')

In [18]:
def download_shp(df):
    for _, row in df.iterrows():
        url = row['Download']
        slug = row['Slug']
        sluglist = []
        folder = os.path.join('data', slug)
        shpfolder = os.path.join('data', slug, 'shp')
        shp = os.path.join('data', slug, 'shp', slug + '.zip')
        
        try:
            response = requests.get(url, timeout = 3)
            response.raise_for_status()
            
            if response.headers['content-type'] == 'application/json; charset=utf-8':
                print('Not a zip file: ', slug)
            else:
                if not os.path.exists(shpfolder):
                    os.makedirs(shpfolder)
                urllib.request.urlretrieve(url, shp)
                with zipfile.ZipFile(shp, 'r') as zip_ref:
                    zip_ref.extractall(shpfolder)
                    zip_ref.close()
                    os.remove(shp)
                    shplist.append(slug)
                    print('Download successfuly: ', slug)
        except requests.exceptions.HTTPError as errh:
            print ('Http Error:',errh)
            
sluglist = download_shp(df_csv)

NameError: name 'shplist' is not defined

In [11]:
sluglist = ['a7a87f76d667497e86321812dd18808f_0', 'fe03b9fce31843c996bfb709cafe6b99_1', 'fe03b9fce31843c996bfb709cafe6b99_0']
df_valid = df_csv[df_csv['Slug'].isin(sluglist)]
df_valid    

,Title,Alternative Title,Description,Language,Creator,Publisher,Genre,Subject,Keyword,Date Issued,...,Provenance,Code,Is Part Of,Status,Accrual Method,Date Accessioned,Rights,Access Rights,Suppressed,Child
0,NaN,Condominium Points,SEMCOG maintains three separate data layers th...,English,Southeast Michigan Council of Governments,State of Michigan,Geospatial data,NaN,Condominiums|BuildingFootprints|Buildings|Condos,2020-06-24T15:39:49.000Z,...,Michigan State University,06a-01,06a-01,Active,ArcGIS Hub,NaN,Public,NaN,False,False
1,NaN,Mdot2021To2025FytpRoad,The 2021-2025 Five-Year Transportation Program...,English,Michigan Department of Transportation,State of Michigan,Geospatial data,NaN,5yp|Five|Year|Transportation|Program|Plan|5YTP...,2021-01-22T20:03:43.000Z,...,Michigan State University,06a-01,06a-01,Active,ArcGIS Hub,NaN,Public,NaN,False,False
2,NaN,Mdot2021To2025FytpBridge,The 2021-2025 Five-Year Transportation Program...,English,Michigan Department of Transportation,State of Michigan,Geospatial data,NaN,5yp|Five|Year|Transportation|Program|Plan|5YTP...,2021-01-22T20:03:43.000Z,...,Michigan State University,06a-01,06a-01,Active,ArcGIS Hub,NaN,Public,NaN,False,False


In [17]:
slug = 'a7a87f76d667497e86321812dd18808f_0'
shpfolder = os.path.join('data', slug, 'shp')

def conversion(inputfile):
    ## convert file to json 
    inputfile = json.loads(inputfile.to_json())
    ## display features properties as dataframe
    df = pd.json_normalize(inputfile['features'])
    return df

geojson = os.path.join('data', slug, slug + '.json')

for file in os.listdir(shpfolder):
    if file.endswith('.shp'):
        gpd_shp = gpd.read_file(os.path.join('data', slug, 'shp', file), driver = 'shapefile').to_crs('EPSG:4326')
        df = conversion(gpd_shp)
df

,id,type,properties.apn,properties.building_i,properties.city_id,properties.condo_id,properties.gid,properties.hiadd1,properties.housing_un,properties.loadd1,properties.nonres_sqf,properties.parcel_id,properties.res_sqft,properties.street1,properties.year_built,properties.zipcode,geometry.type,geometry.coordinates
0,0,Feature,48 002 13 0045 000,1000540,1130,1714213,1,0,1,755,0.0,1196566,1152,RANDOLPH # 125,1969,48167,Point,"[-83.4928088535724, 42.4364397673364]"
1,1,Feature,48 002 13 0046 000,1000540,1130,1714214,2,0,1,755,0.0,1196566,1152,RANDOLPH # 126,1969,48167,Point,"[-83.4928088535724, 42.4364397673364]"
2,2,Feature,48 002 13 0052 000,1000540,1130,1714220,3,0,1,755,0.0,1196566,1152,RANDOLPH # 226,1969,48167,Point,"[-83.4928088535724, 42.4364397673364]"
3,3,Feature,16-11-07-477-021,3133161,3030,3715310,4,0,1,41305,0.0,3298045,1259,DONNA DR,1990,48038,Point,"[-82.9571868395243, 42.6001866737364]"
4,4,Feature,16-11-07-374-026,3132543,3030,3715368,5,0,1,41460,0.0,3324691,1234,REDMOND CT,2001,48038,Point,"[-82.9691987903241, 42.600432560009]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148489,148489,Feature,16-11-21-426-017,3103635,3030,3712949,148490,0,1,37613,0.0,3326027,1500,CHARTER OAKS BLVD,1971,48036,Point,"[-82.9151162440595, 42.5735624662785]"
148490,148490,Feature,16-11-30-455-036,3406252,3030,3710144,148491,0,1,35267,0.0,3278665,1300,SILVER MAPLE DR,2004,48035,Point,"[-82.9587392089351, 42.5554139133695]"
148491,148491,Feature,16-11-19-104-006,3115556,3030,3713710,148492,0,1,38915,0.0,3282539,1600,E WINDMILL PT,1987,48038,Point,"[-82.9699578145488, 42.5819645810188]"
148492,148492,Feature,16-11-17-126-059,3129165,3030,3714957,148493,0,1,17448,0.0,3316717,2100,BRECKENRIDGE DR,2001,48038,Point,"[-82.9448648307101, 42.5976331553257]"
